In [1]:
!pip install tweepy

     |████████████████████████████████| 153kB 8.2MB/s eta 0:00:01


In [3]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords
import random
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
import tweepy
import re
import numpy as np
consumer_key = "fxQ499vYHWpR2RTAErnjBHAds"
consumer_key_secret = "kPJVLOIyKjWOZ5nAGqsT45RaHmtfB7mGs2Y95FjRO10f7jpo6c"
access_token = "1267762463669276672-I1eYvKQ0wnvnIZsu6CpAlPcOtYQV7s"
access_token_secret="ARt2J2yZ8Zt43wnID4FXhbUQR1PfdoWEPozSmlYhVIQMD"

auth = tweepy.OAuthHandler(consumer_key, consumer_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

cs = pd.read_csv('https://raw.githubusercontent.com/tubbytani/IBM-hack-challenge/master/sent.csv?token=ALEGRVIZGATSCY7C25G7ERK7BQQE2',names=['text','sent'],encoding="ISO-8859-1",skiprows=1)
cs=cs.dropna()
sample_size = cs.shape[0]
raw_data = cs['text'].values[:sample_size]
sent = list(cs.sent)
for i in range(len(sent)):
    sent[i] = float(sent[i])
#print(sent[0])    
labels=[]
for i in range((cs.shape[0])):
    if sent[i] > 0:
        labels.append(1)
    elif sent[i] < 0:
        labels.append(-1)
    else:
        labels.append(0)


def preprocess_text(text):
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL', text)
    text = re.sub('@[^\s]+','USER', text)
    text = text.lower().replace("ё", "е")
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
    text = re.sub(' +',' ', text)
    return text.strip()
data = [preprocess_text(t) for t in raw_data]

x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.33, random_state=42)
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])
tuned_parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': [1, 1e-1, 1e-2]
}
from sklearn.metrics import classification_report
score = 'f1_macro'
clf = GridSearchCV(text_clf, tuned_parameters, cv=10, scoring=score)
clf.fit(x_train, y_train)
#print(clf.predict(["worst day ever"]))
print(clf.predict(x_test))
#print(classification_report(y_test, clf.predict(x_test), digits=4))

[-1  1  1 ...  0  0 -1]


In [4]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
wml_credential={
  "apikey": "8mr__gCeXB357Qe609KmAU2lH83xnBYK6U0YY8o-jOXG",
  "iam_apikey_description": "Auto-generated for key 5f894b21-30e2-4106-bd28-eb51cfb5ae93",
  "iam_apikey_name": "Service credentials-1",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/43e65b1dc08a465e99c8e98ed176f188::serviceid:ServiceId-558d7e91-dcea-495f-925f-5ebe5671c63b",
  "instance_id": "a1773f06-5ef9-4763-b50c-d84e115f6a96",
  "url": "https://eu-gb.ml.cloud.ibm.com"
}
client=WatsonMachineLearningAPIClient(wml_credential)
metadata={
    client.repository.ModelMetaNames.AUTHOR_NAME :"Tanisha",
    client.repository.ModelMetaNames.AUTHOR_EMAIL:"tanishabharol.eee1@bvp.edu.in",
    client.repository.ModelMetaNames.NAME:"sentiment_analysis_model"
}
stored_data=client.repository.store_model(clf,meta_props=metadata)
stored_data

2020-07-07 10:54:44,588 - watson_machine_learning_client.metanames - WARNING - 'AUTHOR_EMAIL' meta prop is deprecated. It will be ignored.


{'metadata': {'guid': '05f06ca1-b88e-48b6-8f83-b8ea3a460971',
  'url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/a1773f06-5ef9-4763-b50c-d84e115f6a96/published_models/05f06ca1-b88e-48b6-8f83-b8ea3a460971',
  'created_at': '2020-07-07T10:54:45.086Z',
  'modified_at': '2020-07-07T10:54:45.141Z'},
 'entity': {'runtime_environment': 'python-3.6',
  'learning_configuration_url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/a1773f06-5ef9-4763-b50c-d84e115f6a96/published_models/05f06ca1-b88e-48b6-8f83-b8ea3a460971/learning_configuration',
  'author': {'name': 'Tanisha'},
  'name': 'sentiment_analysis_model',
  'learning_iterations_url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/a1773f06-5ef9-4763-b50c-d84e115f6a96/published_models/05f06ca1-b88e-48b6-8f83-b8ea3a460971/learning_iterations',
  'feedback_url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/a1773f06-5ef9-4763-b50c-d84e115f6a96/published_models/05f06ca1-b88e-48b6-8f83-b8ea3a460971/feedback',
  'latest_version': 

In [6]:
get_uid=client.repository.get_model_uid(stored_data)
get_uid
deploy=client.deployments.create(get_uid,name="NBModel_deployment")
client.deployments.list()
deploy




#######################################################################################

Synchronous deployment creation for uid: '05f06ca1-b88e-48b6-8f83-b8ea3a460971' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='0f285693-fdce-4996-8030-c706bd75046c'
------------------------------------------------------------------------------------------------


------------------------------------  ------------------  ------  --------------  ------------------------  -----------------  -------------
GUID                                  NAME                TYPE    STATE           CREATED                   FRAMEWORK          ARTIFACT TYPE
0f285693-fdce-4996-8030-c706bd75046c  NBModel_deployment  online  DEPLOY_SUCCESS  2020-07-07T10:55:05.853Z  scikit-learn-0.20

{'metadata': {'guid': '0f285693-fdce-4996-8030-c706bd75046c',
  'url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/a1773f06-5ef9-4763-b50c-d84e115f6a96/deployments/0f285693-fdce-4996-8030-c706bd75046c',
  'created_at': '2020-07-07T10:55:05.853Z',
  'modified_at': '2020-07-07T10:55:06.128Z'},
 'entity': {'runtime_environment': 'python-3.6',
  'name': 'NBModel_deployment',
  'scoring_url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/a1773f06-5ef9-4763-b50c-d84e115f6a96/deployments/0f285693-fdce-4996-8030-c706bd75046c/online',
  'deployable_asset': {'name': 'sentiment_analysis_model',
   'url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/a1773f06-5ef9-4763-b50c-d84e115f6a96/published_models/05f06ca1-b88e-48b6-8f83-b8ea3a460971',
   'guid': '05f06ca1-b88e-48b6-8f83-b8ea3a460971',
   'created_at': '2020-07-07T10:55:05.827Z',
   'type': 'model'},
  'description': 'Description of deployment',
  'status_details': {'status': 'DEPLOY_SUCCESS'},
  'model_type': 'scikit-learn-0.20',

In [7]:
scoring_endpoint=client.deployments.get_scoring_url(deploy)
print(scoring_endpoint)


https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/a1773f06-5ef9-4763-b50c-d84e115f6a96/deployments/0f285693-fdce-4996-8030-c706bd75046c/online


In [8]:
client.repository.FunctionMetaNames.show()

------------------  ----  --------
META_PROP NAME      TYPE  REQUIRED
NAME                str   Y
DESCRIPTION         str   N
RUNTIME_UID         str   N
INPUT_DATA_SCHEMA   dict  N
OUTPUT_DATA_SCHEMA  dict  N
TAGS                list  N
------------------  ----  --------


In [10]:
scoring_payload = {"fields": ["text"],"values":[["bad day"]]}
predictions = client.deployments.score(scoring_endpoint,scoring_payload)
print(predictions)
#client.deployments.delete(scoring_endpoint)

SyntaxError: invalid syntax (<ipython-input-10-7a7cd3ace9cc>, line 1)

In [75]:
'''
dep_id = client.deployments.get_uid(deploy)
scoring_payload = {
 client.deployments.ScoringMetaNames.INPUT_DATA: [{  
        "fields": ["text"], 
        "values": ["i am fine"]
     }]
 }
client.deployments.score(deployment_id=dep_id,meta_props=scoring_payload)
'''

'\ndep_id = client.deployments.get_uid(deploy)\nscoring_payload = {\n client.deployments.ScoringMetaNames.INPUT_DATA: [{  \n        "fields": ["text"], \n        "values": ["i am fine"]\n     }]\n }\nclient.deployments.score(deployment_id=dep_id,meta_props=scoring_payload)\n'